In [38]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@7574773b


org.apache.spark.sql.SparkSession@7574773b

# Data Set

In [42]:
case class Salary(depName: String, empNo: Long, name: String, 
    salary: Long, hobby: Seq[String])
import spark.implicits._
val empsalary = Seq(
  Salary("sales",     1,  "Alice",  5000, List("game",  "ski")),
  Salary("personnel", 2,  "Olivia", 3900, List("game",  "ski")),
  Salary("sales",     3,  "Ella",   4800, List("skate", "ski")),
  Salary("sales",     4,  "Ebba",   4800, List("game",  "ski")),
  Salary("personnel", 5,  "Lilly",  3500, List("climb", "ski")),
  Salary("develop",   7,  "Astrid", 4200, List("game",  "ski")),
  Salary("develop",   8,  "Saga",   6000, List("kajak", "ski")),
  Salary("develop",   9,  "Freja",  4500, List("game",  "kajak")),
  Salary("develop",   10, "Wilma",  5200, List("game",  "ski")),
  Salary("develop",   11, "Maja",   5200, List("game",  "farming"))).toDS
empsalary.createTempView("empsalary")
empsalary.show()

+---------+-----+------+------+---------------+
|  depName|empNo|  name|salary|          hobby|
+---------+-----+------+------+---------------+
|    sales|    1| Alice|  5000|    [game, ski]|
|personnel|    2|Olivia|  3900|    [game, ski]|
|    sales|    3|  Ella|  4800|   [skate, ski]|
|    sales|    4|  Ebba|  4800|    [game, ski]|
|personnel|    5| Lilly|  3500|   [climb, ski]|
|  develop|    7|Astrid|  4200|    [game, ski]|
|  develop|    8|  Saga|  6000|   [kajak, ski]|
|  develop|    9| Freja|  4500|  [game, kajak]|
|  develop|   10| Wilma|  5200|    [game, ski]|
|  develop|   11|  Maja|  5200|[game, farming]|
+---------+-----+------+------+---------------+



defined class Salary
empsalary = [depName: string, empNo: bigint ... 3 more fields]


[depName: string, empNo: bigint ... 3 more fields]

 # Finding Avg and total salary over dept

In [51]:
val overCategory = Window.partitionBy('depName)
val df = empsalary
                .withColumn("salaries", collect_list('salary) over overCategory)
                .withColumn("average_salary", (avg('salary) over overCategory).cast("int"))
                .withColumn("total_salary", sum('salary) over overCategory)
        .select("depName", "empNo", "name", "salary", "salaries", "average_salary", "total_salary")
df.show(false)

+---------+-----+------+------+------------------------------+--------------+------------+
|depName  |empNo|name  |salary|salaries                      |average_salary|total_salary|
+---------+-----+------+------+------------------------------+--------------+------------+
|develop  |7    |Astrid|4200  |[4200, 6000, 4500, 5200, 5200]|5020          |25100       |
|develop  |8    |Saga  |6000  |[4200, 6000, 4500, 5200, 5200]|5020          |25100       |
|develop  |9    |Freja |4500  |[4200, 6000, 4500, 5200, 5200]|5020          |25100       |
|develop  |10   |Wilma |5200  |[4200, 6000, 4500, 5200, 5200]|5020          |25100       |
|develop  |11   |Maja  |5200  |[4200, 6000, 4500, 5200, 5200]|5020          |25100       |
|sales    |1    |Alice |5000  |[5000, 4800, 4800]            |4866          |14600       |
|sales    |3    |Ella  |4800  |[5000, 4800, 4800]            |4866          |14600       |
|sales    |4    |Ebba  |4800  |[5000, 4800, 4800]            |4866          |14600       |

overCategory = org.apache.spark.sql.expressions.WindowSpec@220d34ea
df = [depName: string, empNo: bigint ... 5 more fields]


[depName: string, empNo: bigint ... 5 more fields]

In [54]:
val overCategory = Window.partitionBy('depName).orderBy('salary desc)
val df = empsalary
            .withColumn("salaries", collect_list('salary) over overCategory)
            .withColumn("average_salary", (avg('salary) over overCategory).cast("int"))
            .withColumn("total_salary", sum('salary) over overCategory)
    .select("depName", "empNo", "name", "salary", "salaries", "average_salary", "total_salary")
df.show(false)

+---------+-----+------+------+------------------------------+--------------+------------+
|depName  |empNo|name  |salary|salaries                      |average_salary|total_salary|
+---------+-----+------+------+------------------------------+--------------+------------+
|develop  |8    |Saga  |6000  |[6000]                        |6000          |6000        |
|develop  |10   |Wilma |5200  |[6000, 5200, 5200]            |5466          |16400       |
|develop  |11   |Maja  |5200  |[6000, 5200, 5200]            |5466          |16400       |
|develop  |9    |Freja |4500  |[6000, 5200, 5200, 4500]      |5225          |20900       |
|develop  |7    |Astrid|4200  |[6000, 5200, 5200, 4500, 4200]|5020          |25100       |
|sales    |1    |Alice |5000  |[5000]                        |5000          |5000        |
|sales    |3    |Ella  |4800  |[5000, 4800, 4800]            |4866          |14600       |
|sales    |4    |Ebba  |4800  |[5000, 4800, 4800]            |4866          |14600       |

overCategory = org.apache.spark.sql.expressions.WindowSpec@5bbade82
df = [depName: string, empNo: bigint ... 5 more fields]


[depName: string, empNo: bigint ... 5 more fields]

## Rank

![title](https://knockdata.github.io/images/spark-window-function-rank-functions.png)

In [55]:
val overCategory = Window.partitionBy('depName).orderBy('salary desc)
val df = empsalary
            .withColumn("salaries", collect_list('salary) over overCategory)
            .withColumn("rank", rank() over overCategory)
            .withColumn("dense_rank", dense_rank() over overCategory)
            .withColumn("row_number", row_number() over overCategory)
            .withColumn("ntile", ntile(3) over overCategory)
            .withColumn("percent_rank", percent_rank() over overCategory)
        .select("depName", "empNo", "name", "salary", "rank", "dense_rank", "row_number", "ntile", "percent_rank")
df.show(false)

+---------+-----+------+------+----+----------+----------+-----+------------+
|depName  |empNo|name  |salary|rank|dense_rank|row_number|ntile|percent_rank|
+---------+-----+------+------+----+----------+----------+-----+------------+
|develop  |8    |Saga  |6000  |1   |1         |1         |1    |0.0         |
|develop  |10   |Wilma |5200  |2   |2         |2         |1    |0.25        |
|develop  |11   |Maja  |5200  |2   |2         |3         |2    |0.25        |
|develop  |9    |Freja |4500  |4   |3         |4         |2    |0.75        |
|develop  |7    |Astrid|4200  |5   |4         |5         |3    |1.0         |
|sales    |1    |Alice |5000  |1   |1         |1         |1    |0.0         |
|sales    |3    |Ella  |4800  |2   |2         |2         |2    |0.5         |
|sales    |4    |Ebba  |4800  |2   |2         |3         |3    |0.5         |
|personnel|2    |Olivia|3900  |1   |1         |1         |1    |0.0         |
|personnel|5    |Lilly |3500  |2   |2         |2         |2    |

overCategory = org.apache.spark.sql.expressions.WindowSpec@2046c3a6
df = [depName: string, empNo: bigint ... 7 more fields]


[depName: string, empNo: bigint ... 7 more fields]

## lag & lead in a group

lag and lead can be used, when we want to get a relative result between rows. The real values we get are depending on the order.

lag means getting the value from the previous row; lead means getting the value from the next row.

In [57]:
val overCategory = Window.partitionBy('depname).orderBy('salary.desc)
val df = empsalary.withColumn(
  "lead", lead('salary, 1).over(overCategory)).withColumn(
  "lag", lag('salary, 1).over(overCategory)).select(
      "depName", "empNo", "name", "salary", "lead", "lag")
df.show(false)

+---------+-----+------+------+----+----+
|depName  |empNo|name  |salary|lead|lag |
+---------+-----+------+------+----+----+
|develop  |8    |Saga  |6000  |5200|null|
|develop  |10   |Wilma |5200  |5200|6000|
|develop  |11   |Maja  |5200  |4500|5200|
|develop  |9    |Freja |4500  |4200|5200|
|develop  |7    |Astrid|4200  |null|4500|
|sales    |1    |Alice |5000  |4800|null|
|sales    |3    |Ella  |4800  |4800|5000|
|sales    |4    |Ebba  |4800  |null|4800|
|personnel|2    |Olivia|3900  |3500|null|
|personnel|5    |Lilly |3500  |null|3900|
+---------+-----+------+------+----+----+



overCategory = org.apache.spark.sql.expressions.WindowSpec@492b7040
df = [depName: string, empNo: bigint ... 4 more fields]


[depName: string, empNo: bigint ... 4 more fields]

In [58]:
val diff = df.withColumn(
    "higher_than_next", 'salary - 'lead).withColumn(
    "lower_than_previous", 'lag - 'salary)
diff.show()

+---------+-----+------+------+----+----+----------------+-------------------+
|  depName|empNo|  name|salary|lead| lag|higher_than_next|lower_than_previous|
+---------+-----+------+------+----+----+----------------+-------------------+
|  develop|    8|  Saga|  6000|5200|null|             800|               null|
|  develop|   10| Wilma|  5200|5200|6000|               0|                800|
|  develop|   11|  Maja|  5200|4500|5200|             700|                  0|
|  develop|    9| Freja|  4500|4200|5200|             300|                700|
|  develop|    7|Astrid|  4200|null|4500|            null|                300|
|    sales|    1| Alice|  5000|4800|null|             200|               null|
|    sales|    3|  Ella|  4800|4800|5000|               0|                200|
|    sales|    4|  Ebba|  4800|null|4800|            null|                  0|
|personnel|    2|Olivia|  3900|3500|null|             400|               null|
|personnel|    5| Lilly|  3500|null|3900|           

diff = [depName: string, empNo: bigint ... 6 more fields]


[depName: string, empNo: bigint ... 6 more fields]

In [59]:
val diff = df.withColumn(
  "higher_than_next", when('lead.isNull, 0).otherwise('salary - 'lead)).withColumn(
  "lower_than_previous", when('lag.isNull, 0).otherwise('lag - 'salary))
diff.show()

+---------+-----+------+------+----+----+----------------+-------------------+
|  depName|empNo|  name|salary|lead| lag|higher_than_next|lower_than_previous|
+---------+-----+------+------+----+----+----------------+-------------------+
|  develop|    8|  Saga|  6000|5200|null|             800|                  0|
|  develop|   10| Wilma|  5200|5200|6000|               0|                800|
|  develop|   11|  Maja|  5200|4500|5200|             700|                  0|
|  develop|    9| Freja|  4500|4200|5200|             300|                700|
|  develop|    7|Astrid|  4200|null|4500|               0|                300|
|    sales|    1| Alice|  5000|4800|null|             200|                  0|
|    sales|    3|  Ella|  4800|4800|5000|               0|                200|
|    sales|    4|  Ebba|  4800|null|4800|               0|                  0|
|personnel|    2|Olivia|  3900|3500|null|             400|                  0|
|personnel|    5| Lilly|  3500|null|3900|           

diff = [depName: string, empNo: bigint ... 6 more fields]


[depName: string, empNo: bigint ... 6 more fields]

In [60]:
diff.filter('higher_than_next > (lit(2) * 'salary)).show(false)

+-------+-----+----+------+----+---+----------------+-------------------+
|depName|empNo|name|salary|lead|lag|higher_than_next|lower_than_previous|
+-------+-----+----+------+----+---+----------------+-------------------+
+-------+-----+----+------+----+---+----------------+-------------------+



## Running Total

Running Total means adding everything up until the currentRow

In [61]:
val overCategory = Window.partitionBy('depname).orderBy('salary desc)

val running_total = empsalary.withColumn(
  "rank", rank().over(overCategory)).withColumn(
  "costs", sum('salary).over(overCategory)).select(
  "depName", "empNo", "name", "salary", "rank", "costs")
running_total.show(false)

+---------+-----+------+------+----+-----+
|depName  |empNo|name  |salary|rank|costs|
+---------+-----+------+------+----+-----+
|develop  |8    |Saga  |6000  |1   |6000 |
|develop  |10   |Wilma |5200  |2   |16400|
|develop  |11   |Maja  |5200  |2   |16400|
|develop  |9    |Freja |4500  |4   |20900|
|develop  |7    |Astrid|4200  |5   |25100|
|sales    |1    |Alice |5000  |1   |5000 |
|sales    |3    |Ella  |4800  |2   |14600|
|sales    |4    |Ebba  |4800  |2   |14600|
|personnel|2    |Olivia|3900  |1   |3900 |
|personnel|5    |Lilly |3500  |2   |7400 |
+---------+-----+------+------+----+-----+



overCategory = org.apache.spark.sql.expressions.WindowSpec@4779c0a6
running_total = [depName: string, empNo: bigint ... 4 more fields]


[depName: string, empNo: bigint ... 4 more fields]

In [62]:
val overCategory = Window.partitionBy('depname).orderBy('salary desc)
val overRowCategory = Window.partitionBy('depname).orderBy('row_number)

val running_total = empsalary.withColumn(
  "row_number", row_number() over overCategory).withColumn(
  "costs", sum('salary) over overRowCategory).select(
  "depName", "empNo", "name", "salary", "row_number", "costs")
running_total.show(false)

+---------+-----+------+------+----------+-----+
|depName  |empNo|name  |salary|row_number|costs|
+---------+-----+------+------+----------+-----+
|develop  |8    |Saga  |6000  |1         |6000 |
|develop  |10   |Wilma |5200  |2         |11200|
|develop  |11   |Maja  |5200  |3         |16400|
|develop  |9    |Freja |4500  |4         |20900|
|develop  |7    |Astrid|4200  |5         |25100|
|sales    |1    |Alice |5000  |1         |5000 |
|sales    |3    |Ella  |4800  |2         |9800 |
|sales    |4    |Ebba  |4800  |3         |14600|
|personnel|2    |Olivia|3900  |1         |3900 |
|personnel|5    |Lilly |3500  |2         |7400 |
+---------+-----+------+------+----------+-----+



overCategory = org.apache.spark.sql.expressions.WindowSpec@741bfbca
overRowCategory = org.apache.spark.sql.expressions.WindowSpec@2c38290d
running_total = [depName: string, empNo: bigint ... 4 more fields]


[depName: string, empNo: bigint ... 4 more fields]

## Range Frame

We can use range functions to change frame boundary.

    Create with Window.partitionBy on one or more columns
    It usually has orderBy so that the data in the frame is ordered.
    Then followed by rangeBetween or rowsBetween
    Each row will have a corresponding frame
    Frame boundary can be controlled by rangeBetween or rowsBetween
    Aggregate/Window functions can be applied on each row+frame to generate a single value

There are two range window functions, here are the functions definitions

In [64]:
def rowsBetween( start: Long, end: Long): WindowSpec

def rangeBetween(start: Long, end: Long): WindowSpec

Name: Syntax Error.
Message: 
StackTrace: 

both functions accept two parameters, [start, end] all inclusive. The parameters value can be Window.unboundedPreceding, Window.unboundedFollowing, and Window.currentRow. Or a value relative to Window.currentRow, either negtive or positive.

rowsBetween get the frame boundary based on the row index in the window compared to currentRow. here are a few examples and it’s meaning.

range-frame

![title](https://knockdata.github.io/images/spark-window-function-range-frame.png)

In [68]:
val overCategory = Window.partitionBy('depName).rowsBetween(Window.currentRow, 1)

val df = empsalary
        .withColumn("salaries", collect_list('salary) over overCategory)
        .withColumn("total_salary", sum('salary) over overCategory)

df.select("depName", "empNo", "name", "salary", "salaries", "total_salary").show(false)

+---------+-----+------+------+------------+------------+
|depName  |empNo|name  |salary|salaries    |total_salary|
+---------+-----+------+------+------------+------------+
|develop  |7    |Astrid|4200  |[4200, 6000]|10200       |
|develop  |8    |Saga  |6000  |[6000, 4500]|10500       |
|develop  |9    |Freja |4500  |[4500, 5200]|9700        |
|develop  |10   |Wilma |5200  |[5200, 5200]|10400       |
|develop  |11   |Maja  |5200  |[5200]      |5200        |
|sales    |1    |Alice |5000  |[5000, 4800]|9800        |
|sales    |3    |Ella  |4800  |[4800, 4800]|9600        |
|sales    |4    |Ebba  |4800  |[4800]      |4800        |
|personnel|2    |Olivia|3900  |[3900, 3500]|7400        |
|personnel|5    |Lilly |3500  |[3500]      |3500        |
+---------+-----+------+------+------------+------------+



overCategory = org.apache.spark.sql.expressions.WindowSpec@441f6d21
df = [depName: string, empNo: bigint ... 5 more fields]


[depName: string, empNo: bigint ... 5 more fields]

In [70]:
val overCategory = Window.partitionBy('depName).orderBy('salary desc).rowsBetween(Window.currentRow, 1)
val df = empsalary
            .withColumn("salaries", collect_list('salary) over overCategory)
            .withColumn("total_salary", sum('salary) over overCategory)
df.select("depName", "empNo", "name", "salary", "salaries", "total_salary").show(false)

+---------+-----+------+------+------------+------------+
|depName  |empNo|name  |salary|salaries    |total_salary|
+---------+-----+------+------+------------+------------+
|develop  |8    |Saga  |6000  |[6000, 5200]|11200       |
|develop  |10   |Wilma |5200  |[5200, 5200]|10400       |
|develop  |11   |Maja  |5200  |[5200, 4500]|9700        |
|develop  |9    |Freja |4500  |[4500, 4200]|8700        |
|develop  |7    |Astrid|4200  |[4200]      |4200        |
|sales    |1    |Alice |5000  |[5000, 4800]|9800        |
|sales    |3    |Ella  |4800  |[4800, 4800]|9600        |
|sales    |4    |Ebba  |4800  |[4800]      |4800        |
|personnel|2    |Olivia|3900  |[3900, 3500]|7400        |
|personnel|5    |Lilly |3500  |[3500]      |3500        |
+---------+-----+------+------+------------+------------+



overCategory = org.apache.spark.sql.expressions.WindowSpec@648c123
df = [depName: string, empNo: bigint ... 5 more fields]


[depName: string, empNo: bigint ... 5 more fields]

## Median

mean(avg) and median are commonly used in statistics. 

In certain cases median are more robust comparing to mean, since it will filter out outlier values.

![title](https://knockdata.github.io/images/spark-window-function-mean-median.png)

In [74]:
val overCategory = Window.partitionBy('depName).orderBy('salary).rowsBetween(
  Window.unboundedPreceding, Window.unboundedFollowing)

val df = empsalary.withColumn(
  "salaries", collect_list('salary) over overCategory).withColumn(
  "median_salary", element_at('salaries, (size('salaries)/2 + 1).cast("int")))
df.select("depName", "empNo", "name", "salary", "salaries", "median_salary").show(false)

+---------+-----+------+------+------------------------------+-------------+
|depName  |empNo|name  |salary|salaries                      |median_salary|
+---------+-----+------+------+------------------------------+-------------+
|develop  |7    |Astrid|4200  |[4200, 4500, 5200, 5200, 6000]|5200         |
|develop  |9    |Freja |4500  |[4200, 4500, 5200, 5200, 6000]|5200         |
|develop  |10   |Wilma |5200  |[4200, 4500, 5200, 5200, 6000]|5200         |
|develop  |11   |Maja  |5200  |[4200, 4500, 5200, 5200, 6000]|5200         |
|develop  |8    |Saga  |6000  |[4200, 4500, 5200, 5200, 6000]|5200         |
|sales    |3    |Ella  |4800  |[4800, 4800, 5000]            |4800         |
|sales    |4    |Ebba  |4800  |[4800, 4800, 5000]            |4800         |
|sales    |1    |Alice |5000  |[4800, 4800, 5000]            |4800         |
|personnel|5    |Lilly |3500  |[3500, 3900]                  |3900         |
|personnel|2    |Olivia|3900  |[3500, 3900]                  |3900         |

overCategory = org.apache.spark.sql.expressions.WindowSpec@57dd055d
df = [depName: string, empNo: bigint ... 5 more fields]


[depName: string, empNo: bigint ... 5 more fields]

Reference:

https://knockdata.github.io/spark-window-function/